## Compare Population Data Sets

In [16]:
import os
import pandas as pd
import geopandas as gpd
import rasterstats as rs

In [17]:
def zonal_pop_stats(geodf, raster_path, prefix, stat, grpby):
    """

    :param geodf:  Geodataframe with polygons with areas to be summarized
    :param raster_path:   path to population raster
    :param prefix: prefix to indicate population source 
    :param stat: statistics to calculate
    :return:  data frame with calculated stats
    """
    stats = rs.zonal_stats(geodf, raster_path,
                           geojson_out=True, prefix=prefix,
                          stats=[stat],
                          all_touched=True)
    statsdf = gpd.GeoDataFrame.from_features(stats)
    summary = statsdf[[grpby, prefix + stat]].groupby([grpby]).sum()
    summary.reset_index(inplace=True)
    return summary

In [18]:
bounds_gpkg = '../data/work/bounds/bhs_bounds.gpkg'
bounds_lyr = 'bhs_island_groups_32618'
pop_dir = '../data/work/pop'
wp2020_unc = os.path.join(pop_dir,'bhs_ppp_2020_UNadj_32618.tif')
wp2020_con = os.path.join(pop_dir,'bhs_ppp_2020_UNadj_constrained_32618.tif')
wp2010_unc = os.path.join(pop_dir,'bhs_ppp_2010_UNadj_32618.tif')

In [19]:
# Zonal population stats
islands_poly = gpd.read_file(bounds_gpkg, layer=bounds_lyr)

# World Pop 2020, Unconstrained
print("Zonal Stats with ", wp2020_unc)
wp2020_unc_df = zonal_pop_stats(islands_poly, wp2020_unc, 'wp2020_unc_', 'sum', 'island_grp')
print(wp2020_unc_df)

# World Pop 2020, Constrained
print("Zonal Stats with ", wp2020_con)
wp2020_con_df = zonal_pop_stats(islands_poly, wp2020_con, 'wp2020_con_', 'sum', 'island_grp')
print(wp2020_con_df)

# World Pop2010, Unconstrained
print("Zonal Stats with ", wp2010_unc)
wp2010_unc_df = zonal_pop_stats(islands_poly, wp2010_unc, 'wp2010_unc_', 'sum', 'island_grp')
print(wp2010_unc_df)

/Users/arbailey/opt/miniconda3/envs/geoenv/lib/python3.8/site-packages/geopandas/geodataframe.py:577: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


Zonal Stats with  ../data/work/pop/bhs_ppp_2020_UNadj_32618.tif
         island_grp  wp2020_unc_sum
0             Abaco    21479.472656
1   Acklins/Crooked     1038.204712
2            Andros     7054.615234
3     Berry Islands      882.701233
4            Bimini     2026.734009
5        Cat Island     1359.314819
6          Eluthera    11500.934570
7            Exumas    13114.438477
8      Grand Bahama    53533.554688
9      Great Inagua      795.429321
10    Little Inagua       54.770615
11      Long Island     3119.585693
12        Mayaguana      291.212982
13   New Providence   277582.875000
14    Ragged Island       70.611549
15          Rum Cay      119.700272
16     San Salvador      887.271179
Zonal Stats with  ../data/work/pop/bhs_ppp_2020_UNadj_constrained_32618.tif
         island_grp  wp2020_con_sum
0             Abaco    21634.251953
1   Acklins/Crooked     1062.821777
2            Andros     6960.700195
3     Berry Islands      855.954712
4            Bimini     1968.379

In [20]:
print("Combining outputs")
metrics = pd.concat([wp2020_unc_df, wp2020_con_df.iloc[:,-1:], wp2010_unc_df.iloc[:,-1:]], axis=1)

print(metrics)
print(metrics.describe())

Combining outputs
         island_grp  wp2020_unc_sum  wp2020_con_sum  wp2010_unc_sum
0             Abaco    21479.472656    21634.251953    17311.779297
1   Acklins/Crooked     1038.204712     1062.821777      926.218750
2            Andros     7054.615234     6960.700195     7637.671875
3     Berry Islands      882.701233      855.954712      817.881592
4            Bimini     2026.734009     1968.379150     1839.238770
5        Cat Island     1359.314819     1302.778442     1549.326904
6          Eluthera    11500.934570    11786.451172    11821.157227
7            Exumas    13114.438477    13695.203125     7113.812012
8      Grand Bahama    53533.554688    53477.917969    51600.218750
9      Great Inagua      795.429321      845.474487      881.864319
10    Little Inagua       54.770615        0.000000       71.171936
11      Long Island     3119.585693     3156.461670     3184.705078
12        Mayaguana      291.212982      302.286591      287.183472
13   New Providence   277582.8

In [21]:
metrics.to_csv(os.path.join(pop_dir,'wp_2010_2020_compare.csv'))